- sportsnaviプロ野球ページからのwebスクレイピングスクリプト

In [ ]:
import os
import re
import requests
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
team_replace= {'福岡': 'ソフトバンク',  '東北': '楽天','埼玉': '西武', 'オリ': 'オリックス','千葉': 'ロッテ', '北海': '日本ハム',  
 '読売': '巨人', '横浜': 'DeNA', '東京': ' ヤクルト', '広島': '広島', '中日': '中日', '阪神': '阪神'}

**function**

In [ ]:
# dateで受け取った日に開催された各試合の出場成績のリンクをリスト形式で返す
def fetch_game_links(date):
    params = { 'date': date }
    schedule_page = requests.get('https://baseball.yahoo.co.jp/npb/schedule', params=params)
    soup_schedule = BeautifulSoup(schedule_page.text, 'html.parser')
    game_link_elms = soup_schedule.find_all('a', class_='bb-score__content')
    game_links = list(map(lambda x: x['href'].replace('index', 'stats'), game_link_elms))
    detail_links = list(map(lambda x: x['href'].replace('index', 'score'), game_link_elms))
    text_links = list(map(lambda x: x['href'].replace('index', 'text'), game_link_elms))
    return game_links, detail_links, text_links

def fetch_score_stats(game_link, dat, id_):
    pacific = ['ソフトバンク',  '楽天', '西武',  'オリックス', 'ロッテ', '日本ハム']
    central = ['広島', '巨人', 'DeNA', 'ヤクルト',  '中日',  '阪神']
    
    game_page = requests.get(game_link)
    soup_game = BeautifulSoup(game_page.text, 'html.parser')
    
    score_stats_rows = soup_game.find_all('td', class_='bb-gameScoreTable__data')
    result_kind_rows = ["チーム"] + [str(i) for i in range(1,13)]
    result_kind_rows.append("日付")
    result_kind_rows.append("ID")
    
    stats_list = [result_kind_rows]
    stats = [score_stats_rows[0].text.replace('\n', '')]
    count = 1

    for ind, row in enumerate(score_stats_rows[1:]):
        ele = row.text.replace('\n', '') if len(row.text.replace('\n', '')) > 0 else 'X'
        
        if ele in pacific + central:
            stats += ["X"] * (13 - len(stats))
            stats.append(dat)
            stats.append(id_)
            stats_list.append(stats)
            stats = []
        elif ele != 'X':
            if ele[-1] == 'X':
                ele = int(ele[:-1])
            else:
                ele = int(ele)
    
        stats.append(ele)

    stats += ["X"] * (13 - len(stats))
    stats.append(dat)
    stats.append(id_)
    stats_list.append(stats)
        
    return stats_list

# game_linkで受け取ったリンク先の野手成績をリスト形式で返す
def fetch_batter_stats(game_link, dat, id_):
    flg = False
    game_page = requests.get(game_link)
    soup_game = BeautifulSoup(game_page.text, 'html.parser')
    
    result_kind_rows = soup_game.find_all('p', class_='bb-statsTable__headLabel')
    result_kind_rows = [i.text for i in result_kind_rows]
    result_kind_rows = result_kind_rows[:len(result_kind_rows)//2]
    result_kind_rows.insert(1, "名")
    result_kind_rows.append("自")
    result_kind_rows.append("相")
    result_kind_rows.append("H/V")
    result_kind_rows.append("日付")
    result_kind_rows.append("ID")
    
    team_name_rows = soup_game.find_all('h1', class_='bb-head03__title')
    team_name_rows = [i.text for i in team_name_rows]
    
    batter_stats_rows = soup_game.find_all('tr', class_='bb-statsTable__row')

    stats_list = [result_kind_rows]
    # 野手成績
    for row in batter_stats_rows:
        if len(row.find_all('p', class_='bb-statsTable__headLabel')) and flg:
            my = team_name_rows[1][:2]
            enemy = team_name_rows[0][:2]
            team = "H"
        if len(row.find_all('p', class_='bb-statsTable__headLabel')) and not flg:
            my = team_name_rows[0][:2]
            enemy = team_name_rows[1][:2]
            team = "V"
            flg = True
    
        stats_html = row.find_all('td', class_='bb-statsTable__data')
        if stats_html:
            stats = list(map(lambda x: x.text, stats_html))[:14]
            stats[0] = re.sub("[()打走]", '', stats[0])
            stats[0] = '指' if len(stats[0]) == 0 else stats[0][:len(stats[0])] # 最初に出場したポジションのみに変換（代打のみの場合は指名打者扱い）
            stats[3:] = list(map(int, stats[3:])) # 野手成績をintに変換
            stats.append(my)
            stats.append(enemy)
            stats.append(team)
            stats.append(dat)
            stats.append(id_)
            stats_list.append(stats)
    return stats_list

# game_linkで受け取ったリンク先の投手成績をリスト形式で返す
def fetch_pitcher_stats(game_link, dat, id_):
    flg = False
    count = 0
    game_page = requests.get(game_link)
    soup_game = BeautifulSoup(game_page.text, 'html.parser')

    result_kind_rows = soup_game.find_all('th', class_='bb-scoreTable__head')
    result_kind_rows = [i.text for i in result_kind_rows]
    result_kind_rows = result_kind_rows[:len(result_kind_rows)//2]
    result_kind_rows[0] = "位"
    result_kind_rows.append("自")
    result_kind_rows.append("相")
    result_kind_rows.append("H/V")
    result_kind_rows.append("日付")
    result_kind_rows.append("ID")
    
    team_name_rows = soup_game.find_all('h1', class_='bb-head03__title')
    team_name_rows = [i.text for i in team_name_rows]
    
    pitcher_stats_rows = soup_game.find_all(['tr', 'h1'], class_=['bb-scoreTable__row', 'bb-head03__title'])
    words = ["北海道", "東北", "埼玉", "千葉ロッテ", "オリックス", "ソフトバンク", "横浜", "東京", "読売", "中日", "阪神", "広島"]
    stats_list = [result_kind_rows]
    # 投手成績
    for row in pitcher_stats_rows:
        if any([s in row.text for s in words]) and flg:
            my = team_name_rows[1][:2]
            enemy = team_name_rows[0][:2]
            team = "H"
            count = 0
        if any([s in row.text for s in words]) and not flg:
            my = team_name_rows[0][:2]
            enemy = team_name_rows[1][:2]
            team = "V"
            flg = True
            
        stats_html = row.find_all('td', class_='bb-scoreTable__data')
        if stats_html:
            stats = list(map(lambda x: x.text, stats_html))[:14]
            stats[0] = '先発' if count == 0 else '中継'
            stats[1] = stats[1].replace('\n', '')
            stats[2] = stats[2].replace('\n', '')
            stats[3] = float(stats[3]) # 投球回をfloatに変換
            stats[4:] = list(map(int, stats[4:])) # 投手成績をintに変換
            stats.append(my)
            stats.append(enemy)
            stats.append(team)
            stats.append(dat)
            stats.append(id_)
            stats_list.append(stats)
            count += 1
    return stats_list


# 各試合のイニングへのリンクをリストで取得
def get_inning_links(detail_link):
    game_page = requests.get(detail_link)
    soup_game = BeautifulSoup(game_page.text, 'html.parser')
    
    result_rows = soup_game.find_all('td', class_="bb-gameScoreTable__data")
    link_to_each_inning = []
    
    for row in result_rows:
        count = 100
        if row.find_all('a', class_='bb-gameScoreTable__score'):
            text = row.find_all('a', class_='bb-gameScoreTable__score')[0].get('href')
            inning = int(text[-6]) if int(text[-7]) == 0 else int(text[-7:-5])
            
            if text[-3:]== "000":
                text = text[:-3] + str(count)
            link_to_each_inning.append([inning, "https://baseball.yahoo.co.jp"+text])
            #print("https://baseball.yahoo.co.jp"+text)
            
            while True:  
                count += 100
                if count >= 1000:
                    text = text[:-4] + str(count)
                else:
                    text = text[:-3] + str(count)
                url = "https://baseball.yahoo.co.jp"+text
                res = requests.get(url)
                if res.status_code != 404:
                    link_to_each_inning.append([inning, "https://baseball.yahoo.co.jp"+text])
                else:
                    break
    
    return sorted(link_to_each_inning)

# 1球データを手に入れる
def fetch_details(inning_links, dat, id_):
    N = 20
    people_col = ["投手", "投", "打者", "打", "T/B", "投手チーム", "打者チーム"]
    balltype_col = ["type_"+str(i) for i in range(1, N+1)]
    speed_col = ["speed_"+str(i) for i in range(1, N+1)]
    result_col = ["result_"+str(i) for i in range(1, N+1)]
    element_list = [people_col + balltype_col + speed_col + result_col + [ "イニング", "日付", "ID"]]
    
    for inning, link_ in inning_links:
        game_page = requests.get(link_)
        soup_game = BeautifulSoup(game_page.text, 'html.parser')
        
        if inning == 1:
            title = soup_game.findAll('h1', {'class':'bb-head01__title'}) 
            title= [i.text for i in title] 
            title = re.sub(r"[¥\n]{1,}", " ", title[0])   
            title = re.sub(r" ", '', title)
            title = title.split("vs.")
            
        table = soup_game.find_all('a', target="npb_live") 
        
        b_or_t = soup_game.find_all('em') 
        dominant_hand = soup_game.find_all('td', class_="dominantHand") 
        dominant_hand = [i.text for i in dominant_hand]
        
        if "表" in b_or_t[0].text:
            #打者また投手
            p1 = table[1].text 
            p2 = table[3].text
            people = [p1, dominant_hand[0], p2, dominant_hand[1]] 
            people.append("T")
            people.append(title[0])
            people.append(title[1])
        else:
            #打者また投手
            p1 = table[4].text 
            p2 = table[2].text
            people = [p1, dominant_hand[1], p2, dominant_hand[0]] 
            people.append("B")
            people.append(title[1])
            people.append(title[0])
        
        # 球種
        balltype = soup_game.find_all('td', class_="bb-splitsTable__data--ballType")
        balltype = [ele.text for ele in balltype]
        balltype += [np.nan for _ in range(N-len(balltype))]
    
        # 球速
        speed = soup_game.find_all('td', class_="bb-splitsTable__data--speed")
        speed = [ele.text for ele in speed]
        speed+= [np.nan for _ in range(N-len(speed))]
        
        result = soup_game.find_all('td', class_="bb-splitsTable__data--result")
        result = [ele.text.replace('\n', '').replace(' ', '') for ele in result]
        result += [np.nan for _ in range(N-len(result))]
        
        element = people + balltype + speed + result + [inning, dat, id_]
        
        element_list.append(element)
            
    return element_list

def connect_data(list_):
    main_df = pd.DataFrame(list_[0])
    for df_ in list_[1:]:
        df_ = pd.DataFrame(df_)
        main_df = pd.concat([main_df, df_.iloc[1:]], axis=0)
    main_df.columns = main_df.iloc[0]
    main_df = main_df.iloc[1:].reset_index(drop=True)
    main_df["日付"] = pd.to_datetime(main_df["日付"])
    main_df["week"] = (main_df["日付"] - pd.to_datetime('2022-03-25')).dt.days // 7
    return main_df

def make_score_result(df):
    columns =["1","2","3","4","5","6","7","8","9","10","11","12"]
    df[columns] = df[columns].replace("X", np.nan)
    df = df[~np.all(df[columns].isnull(), axis=1)].reset_index(drop=True) #試合中止分を削除
    df["total"] = df[columns].sum(axis=1).astype(int)

    visitor_wl = np.where(df["total"][0::2].values > df["total"][1::2].values, "W", "L")
    home_wl = np.where(df["total"][0::2].values < df["total"][1::2].values, "W", "L")
    draw = np.where(df["total"][0::2].values == df["total"][1::2].values, "D",  "K")

    all_d = np.empty(visitor_wl.shape[0] + visitor_wl.shape[0],dtype=str)
    all_d[0::2] = draw
    all_d[1::2] = draw

    all_wl = np.empty(visitor_wl.shape[0] + visitor_wl.shape[0],dtype=str)
    all_wl[0::2] = visitor_wl
    all_wl[1::2] = home_wl

    for i in range(all_wl.shape[0]):
        if all_d[i] == "D":
            all_wl[i] = "D"

    df["result"] = all_wl
    return df

**data collection**

In [ ]:
if __name__ == '__main__':
    for day in tqdm(range(14,31)):
        d_today = datetime.date(2022,5,day)
        
        game_links, detail_links, text_links = fetch_game_links(d_today)
        batter_list = []
        pitcher_list = []
        score_list = []
    
        detail_list = []
        
        if len(game_links) > 0:
            for id_, game_link in enumerate(game_links):
                detail_link = detail_links[id_]
                # 野手成績取得
                batter_stats = fetch_batter_stats(game_link, str(d_today), id_)
                # 投手成績取得
                pitcher_stats = fetch_pitcher_stats(game_link, str(d_today), id_)
                # スコア取得
                score_stats = fetch_score_stats(game_link, str(d_today), id_)
                # 1球データ取得
                inning_links = get_inning_links(detail_link) 
                detail_stats = fetch_details(inning_links, str(d_today), id_)
        
                batter_list.append(batter_stats) 
                pitcher_list.append(pitcher_stats)
                score_list.append(score_stats)
                detail_list.append(detail_stats)
    
            batter_df = connect_data(batter_list)
            pitcher_df = connect_data(pitcher_list)
            score_df = connect_data(score_list)
            score_df = make_score_result(score_df)
            detail_df = connect_data(detail_list)
    
            batter_df["自"] = batter_df["自"].map(team_replace)
            batter_df["相"] = batter_df["相"].map(team_replace)

            pitcher_df["自"] = pitcher_df["自"].map(team_replace)
            pitcher_df["相"] = pitcher_df["相"].map(team_replace)
    
            if str(d_today) not in os.listdir("./data/"):
                os.makedirs("./data/"+str(d_today))
        
            batter_df.to_csv("./data/"+str(d_today)+"/batter.csv", index=False)
            pitcher_df.to_csv("./data/"+str(d_today)+"/pitcher.csv", index=False)
            score_df.to_csv("./data/"+str(d_today)+"/score.csv", index=False)
            detail_df.to_csv("./data/"+str(d_today)+"/detail.csv", index=False)

参考
- https://qiita.com/Jun-T/items/6a641b6e28c487127484
- https://gammasoft.jp/support/solutions-of-requests-get-failed/
- https://tonari-it.com/python-response-status-code/

In [ ]:
# dateで受け取った日に開催された各試合の出場成績のリンクをリスト形式で返す
def fetch_game_links(date):
    params = { 'date': date }
    schedule_page = requests.get('https://baseball.yahoo.co.jp/npb/schedule', params=params)
    soup_schedule = BeautifulSoup(schedule_page.text, 'html.parser')
    game_link_elms = soup_schedule.find_all('a', class_='bb-score__content')
    game_links = list(map(lambda x: x['href'].replace('index', 'stats'), game_link_elms))
    detail_links = list(map(lambda x: x['href'].replace('index', 'score'), game_link_elms))
    text_links = list(map(lambda x: x['href'].replace('index', 'text'), game_link_elms))
    top_links = list(map(lambda x: x['href'].replace('index', 'top'), game_link_elms))

    return game_links, detail_links, text_links, top_links

def fetch_text_info(text_link, top_link, dat, id_):
    pacific = ['ソフトバンク',  '楽天', '西武',  'オリックス', 'ロッテ', '日本ハム']
    central = ['広島', '巨人', 'DeNA', 'ヤクルト',  '中日',  '阪神']
    
    # text情報
    game_page = requests.get(text_link)
    soup_game = BeautifulSoup(game_page.text, 'html.parser')
    tmp = soup_game.find_all('section', class_='bb-liveText')    
    
    # 選手情報
    top_page = requests.get(top_link)
    top_page = BeautifulSoup(top_page.text, 'html.parser')
    print(top_page)
    
    first_list = []
    second_list = []
    
    # 両チームの選手一覧
    
    for index_, contents in enumerate(tmp[1:]):
        
        order = contents.find_all('span', class_='bb-liveText__order')
        team = contents.find_all('p', class_='bb-liveText__detail')
        player = contents.find_all('a', class_='bb-liveText__player')
        state = contents.find_all('span', class_='bb-liveText__state')
        result = contents.find_all('p', class_='bb-liveText__summary')
        order = [i.text for i in order]
        team = [i.text.split("の")[0] for i in team] * len(order)
        player = [i.text for i in player]
        state = [i.text for i in state]
    
    
        # 守備側の選手がresultに含まれているので、それらをplayerから除去
        
        # 代打では余計に選手が含まれるので除去
        pinch_hitter_index = [i for i in range(len(order)) if order[i] == "代打"]
        remove_batter_index = []
        for i in pinch_hitter_index:
            remove_batter_index.append(i+1)
            remove_batter_index.append(i+2)
        player = [player[i] for i in range(len(player)) if i not in remove_batter_index]
        
        for i in range(len(result)):
            mod_result = re.sub(r" ", '', result[i].text)
            mod_result = re.sub(r"[¥\n]{2,}", " ", mod_result)
            mod_result = re.sub(r"[¥\n]{1,}", " ", mod_result)     
            result[i] = mod_result
        result = [i for i in result if "交代"not in i 
                                                  and "マウンド" not in i 
                                                  and "けん制" not in i 
                                                  and "盗塁" not in i 
                                                  and "代打" not in i
                                                  and "変更" not in i 
                                                  and "代走" not in i 
                                                 and "試合終了" not in i
                     ]
            
        print(order, team, player, state, result)
        assert len(np.unique([len(order), len(team), len(player), len(state), len(result)])) == 1
            
        for i in range(len(result)):
            element = [order[i],  team[i], player[i], state[i], result[i], dat, id_]
        
            if index_ % 2 == 0:
                first_list.append(element)
            else:
                second_list.append(element)
        
    return first_list #, second_list

d_today = datetime.date(2022,5,6)
game_links, detail_links, text_links, top_links = fetch_game_links(d_today)

for id_, game_link in enumerate(game_links[:1]):
    text_link = text_links[id_]
    top_link = top_links[id_]
    print(id_, text_link)
    fetch_text_info(text_link, top_link, d_today, id_)